In [39]:
from curl_cffi import requests
import parsel
import pandas as pd

# from rich import print
import parsel.selector
from rich.progress import Progress
import re

with open('./Academic staff - The University of Sydney.html', 'r') as f:
    html = f.read()
s = parsel.Selector(html)
urls = s.xpath('//*[@id="staffListContent"]//a/@href').getall()
names = s.xpath('//*[@id="staffListContent"]//a/text()').getall()
df = pd.DataFrame({'name': names, 'url': urls})
df_surname = pd.read_csv('./姓氏列表/常见姓氏列表.csv', header=None, names=['surname'])
# surname是否在name的字符中
# 获取姓氏列表，去除缺失值并去重
surnames = df_surname['surname'].str.strip().dropna().unique().tolist()

# 创建正则表达式模式，使用管道符 '|' 分隔
pattern = r'\b(?:' + '|'.join(surnames) + r')\b'

df['contains_surname'] = df['name'].str.contains(pattern, case=False, na=False)
df[df['contains_surname']].to_excel('./悉尼大学/url.xlsx', index=False)

# 显示结果

In [44]:
import requests

cookies = {
    'UBT_VID': '1731850229796.955eCGBHB9sB',
    'MKT_CKID': '1731850230665.a8956.tkdh',
    'GUID': '09031035310415008038',
    'Session': 'smartlinkcode=U130678&smartlinklanguage=zh&SmartLinkKeyWord=&SmartLinkQuary=&SmartLinkHost=',
    'Union': 'AllianceID=4899&SID=130678&OUID=&createtime=1731850231&Expires=1732455031488',
    '_RF1': '116.28.140.26',
    '_RSG': '1FKhIVCjag04sqgcHuIkyA',
    '_RDG': '2865e40112266829e43f52ab337f4c83a3',
    '_RGUID': '0aa4b4c7-d2ad-43c1-af2e-4b8466ab4d84',
    'MKT_Pagesource': 'PC',
    'nfes_isSupportWebP': '1',
    '_jzqco': '%7C%7C%7C%7C%7C1.1500767260.1731850230663.1731850230663.1731850246203.1731850230663.1731850246203.0.0.0.2.2',
    '_bfa': '1.1731850229796.955eCGBHB9sB.1.1731850229816.1731850249682.1.2.290510',
}

headers = {
    'accept': '*/*',
    'accept-language': 'zh-CN,zh;q=0.9',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    # 'cookie': 'UBT_VID=1731850229796.955eCGBHB9sB; MKT_CKID=1731850230665.a8956.tkdh; GUID=09031035310415008038; Session=smartlinkcode=U130678&smartlinklanguage=zh&SmartLinkKeyWord=&SmartLinkQuary=&SmartLinkHost=; Union=AllianceID=4899&SID=130678&OUID=&createtime=1731850231&Expires=1732455031488; _RF1=116.28.140.26; _RSG=1FKhIVCjag04sqgcHuIkyA; _RDG=2865e40112266829e43f52ab337f4c83a3; _RGUID=0aa4b4c7-d2ad-43c1-af2e-4b8466ab4d84; MKT_Pagesource=PC; nfes_isSupportWebP=1; _jzqco=%7C%7C%7C%7C%7C1.1500767260.1731850230663.1731850230663.1731850246203.1731850230663.1731850246203.0.0.0.2.2; _bfa=1.1731850229796.955eCGBHB9sB.1.1731850229816.1731850249682.1.2.290510',
    'cookieorigin': 'https://you.ctrip.com',
    'origin': 'https://you.ctrip.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://you.ctrip.com/',
    'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
}

params = {
    '_fxpcqlniredt': '09031035310415008038',
    'x-traceID': '09031035310415008038-1731850436060-2059472',
}

json_data = {
    'arg': {
        'channelType': 2,
        'collapseType': 0,
        'commentTagId': 0,
        'pageIndex': 8,
        'pageSize': 10,
        'poiId': 10520807,
        'sourceType': 1,
        'sortType': 3,
        'starType': 0,
    },
    'head': {
        'cid': '09031035310415008038',
        'ctok': '',
        'cver': '1.0',
        'lang': '01',
        'sid': '8888',
        'syscode': '09',
        'auth': '',
        'xsid': '',
        'extension': [],
    },
}

response = requests.post(
    'https://m.ctrip.com/restapi/soa2/13444/json/getCommentCollapseList',
    params=params,
    # cookies=cookies,
    headers=headers,
    json=json_data,
)
with open('./ctrip.html', 'w') as f:
    f.write(response.text)